<a href="https://colab.research.google.com/github/seanreed1111/colab-demos/blob/master/streamlit_db_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/colinmcnamara/austin_langchain/blob/main/labs/LangChain_101/101-1-streamlit_streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# upload allconfig.json, dbconfig.json, DDL_for_LLM_upload.sql

In [46]:
%pip install -qU langchain langchain_community langchain_openai streamlit loguru sqlalchemy pyodbc

In [47]:
!bash install_drivers.sh > /dev/null

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0  10779      0 --:--:-- --:--:-- --:--:-- 10802
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0    661      0 --:--:-- --:--:-- --:--:--   666


In [ ]:
# ADD in the Callback Handler for the agent
# - https://github.com/langchain-ai/langchain/issues/16489
# - https://python.langchain.com/docs/use_cases/sql/agents
https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.sql.base.create_sql_agent.html
# - https://python.langchain.com/docs/integrations/toolkits/sql_database#use-sqldatabasetoolkit-within-an-agenthttps://python.langchain.com/docs/expression_language/cookbook/sql_db


In [58]:
%%writefile streaming_app.py
## adds openai-tools to agent tool creation BETTER, but still doesn't work

# experiment-5-chat-and-execute-sql-with-WAB FAILURE
# reference docs https://python.langchain.com/docs/integrations/toolkits/sql_database#use-sqldatabasetoolkit-within-an-agenthttps://python.langchain.com/docs/expression_language/cookbook/sql_db

#https://python.langchain.com/docs/expression_language/cookbook/sql_db
import streamlit as st
from pathlib import Path
import os, json
from langchain_community.chat_models.azure_openai import AzureChatOpenAI #deprecated class, fix later
from langchain.agents import create_sql_agent
from langchain.sql_database import SQLDatabase #from langchain_community.utilities.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ChatMessage
)
from sqlalchemy import create_engine
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import urllib
from loguru import logger

LANGCHAIN_PROJECT = "experiment-5-chat-and-execute-sql-with-WAB"
st.set_page_config(page_title=LANGCHAIN_PROJECT, page_icon="")
st.title(LANGCHAIN_PROJECT)

def load_schema_from_file(file):
    try:
        with open(file, 'r') as f:
            schema = f.read()
        assert schema is not None
        return schema

    except Exception as e:
        logger.error(e)

def run_azure_config():
    config_dir = Path.cwd()
    openai_config_file_path = config_dir / "allconfig.json"
    config_files = [openai_config_file_path]
    config = {}
    for file in config_files:
        with open(file) as json_config:
            config.update(json.load(json_config))
    for k in config:
        os.environ[k] = config[k]

    os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

def get_wab_connection_string(db_config_file="dbconfig.json"):
    driver= '{ODBC Driver 18 for SQL Server}'
    db_config_path = Path.cwd() / db_config_file

    with open(db_config_path) as json_file:
        dbconfig = json.load(json_file)

    server = dbconfig['server']
    database = dbconfig['database']
    uid = dbconfig['username']
    pwd = dbconfig['password']
    port = int(dbconfig.get("port",1433))
    pyodbc_connection_string = f"DRIVER={driver};SERVER={server};PORT={port};DATABASE={database};UID={uid};PWD={pwd};Encrypt=yes;Connection Timeout=30;READONLY=True;"
    params = urllib.parse.quote_plus(pyodbc_connection_string)
    sqlalchemy_connection_string = f"mssql+pyodbc:///?odbc_connect={params}"
    return sqlalchemy_connection_string


run_azure_config()


with st.sidebar:
    text = st.text("SQL QUERIES WILL GO HERE")
    SCHEMA_FILEPATH  = (Path.cwd()) / "DDL_for_LLM_upload.sql"
    uploaded_schema = f"{load_schema_from_file(SCHEMA_FILEPATH)}"
    # schema = st.text(f"{uploaded_schema}")


# User inputs
# radio_opt = ["Use sample Chinook database", "Connect to WAB Database"]
# selected_opt = st.sidebar.radio(label="Choose suitable option", options=radio_opt)
# if radio_opt.index(selected_opt) == 1:
#     db_uri = st.sidebar.text_input(
#         label="Database URI", placeholder="azsqldb-genai-dataanalytics-sb"
#     )
# else:
#     db_uri = "CHINOOKDB"

# Setup agent
llm = AzureChatOpenAI(
            temperature=0,
            streaming=True,
            max_tokens=1000,
            azure_deployment=os.environ["AZURE_OPENAI_API_DEPLOYMENT_NAME_GPT35"],
            azure_endpoint=os.environ["AZURE_OPENAI_API_ENDPOINT"],
            model_name=os.environ["MODEL_NAME_GPT35"],
            openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
            request_timeout=45,
            verbose=True,
            #callbacks=[stream_handler] PUSH THE HANDLING OF CALLBACKS TO THE AGENT
        )


@st.cache_resource(ttl="2h")
def get_db_engine(db_config_file="dbconfig.json"):

    def get_wab_connection_string(db_config_file=db_config_file):
        driver= '{ODBC Driver 18 for SQL Server}'
        db_config_path = Path.cwd() / db_config_file

        with open(db_config_path) as json_file:
            dbconfig = json.load(json_file)

        server = dbconfig['server']
        database = dbconfig['database']
        uid = dbconfig['username']
        pwd = dbconfig['password']
        port = int(dbconfig.get("port",1433))
        pyodbc_connection_string = f"DRIVER={driver};SERVER={server};PORT={port};DATABASE={database};UID={uid};PWD={pwd};Encrypt=yes;Connection Timeout=30;READONLY=True;"
        params = urllib.parse.quote_plus(pyodbc_connection_string)
        sqlalchemy_connection_string = f"mssql+pyodbc:///?odbc_connect={params}"
        return sqlalchemy_connection_string

    return SQLDatabase.from_uri(database_uri=get_wab_connection_string())

db=get_db_engine()
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type="openai-tools",
)

with st.sidebar:
    dialect = st.text(f"dialect is {db.dialect}")
    table_names = st.text(f"{db.get_usable_table_names()}")
    q = """
    SELECT TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA = 'sandbox'
    ORDER BY TABLE_NAME;
    """
    sample = st.text('sample query -- db.run(q)')
    result = st.text(f"{db.run(q)}")
    table_info = st.text(f"table info is {db.get_table_info_no_throw()}")

    context = st.text(f"Your context is {toolkit.get_context()}")
    tools = st.text(f" Your tools are {toolkit.get_tools()}")




if "messages" not in st.session_state or st.sidebar.button("Clear message history"):
    # see https://blog.langchain.dev/llms-and-sql/
    system_message = f"""
    You are an expert at writing Mircosoft SQL database queries and T-SQL code.
      When asked to write SQL queries use the following schema
     \n\n\n
    {uploaded_schema}
     \n\n\n
     After writing a query, score its estimated accuracy in answering the
      user's question on a scale of 1-5, with 1 the lowest score and 5 the
      highest possible. Respond with the query and the accuracy score. If you give
      an accuracy score of 1 or 2, briefly state your reason.
     """
    st.session_state["messages"] = [{"role": "system", "content":system_message},
                                    {"role": "assistant", "content": "How can I help you?"}
                                    ]
for msg in st.session_state.messages:
    if msg["role"] != "system":
        st.chat_message(msg["role"]).write(msg["content"])

user_query = st.chat_input(placeholder="Ask me anything!")

if user_query:
    st.session_state.messages.append({"role": "user", "content": user_query})
    st.chat_message("user").write(user_query)

    with st.chat_message("assistant"):
        st_cb = StreamlitCallbackHandler(st.container())
        response = agent.invoke(user_query, callbacks=[st_cb])
        st.session_state.messages.append({"role": "assistant", "content": response})
        st.write(response)


Overwriting streaming_app.py


In [60]:
%%writefile streaming_app.py
# experiment-5-chat-and-execute-sql-with-WAB FAILURE
# reference docs https://python.langchain.com/docs/integrations/toolkits/sql_database#use-sqldatabasetoolkit-within-an-agenthttps://python.langchain.com/docs/expression_language/cookbook/sql_db

#https://python.langchain.com/docs/expression_language/cookbook/sql_db


#####
'''
Your context is {'table_info': '\nCREATE TABLE [TELEPHONE_TYPE] (\n\tcolumn1 NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, \n\tcolumn2 NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL\n)\n\n/*\n3 rows from TELEPHONE_TYPE table:\ncolumn1\tcolumn2\nL\tLandline\nM\tMobile\nF\tFax\n*/', 'table_names': 'TELEPHONE_TYPE'}

Your tools are [QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a9b5f0a6680>), InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a9b5f0a6680>), ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a9b5f0a6680>), QuerySQLCheckerTool(description='Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a9b5f0a6680>, llm=AzureChatOpenAI(verbose=True, client=<openai.resources.chat.completions.Completions object at 0x7a9b6f50cac0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7a9b6f64a290>, model_name='gpt-35-turbo', temperature=0.0, openai_api_key='d7ca6573f74d4b97a97b9f8e58e4d75b', openai_proxy='', request_timeout=45.0, streaming=True, max_tokens=1000, azure_endpoint='https://opnai-genai-dataanlytics-sb.openai.azure.com/', deployment_name='mdl_gpt35_turbo_1106-wmp_genai_dataanalytics-sb', openai_api_version='2024-03-01-preview', openai_api_type='azure'), llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['dialect', 'query'], template='\n{query}\nDouble check the {dialect} query above for common mistakes, including:\n- Using NOT IN with NULL values\n- Using UNION when UNION ALL should have been used\n- Using BETWEEN for exclusive ranges\n- Data type mismatch in predicates\n- Properly quoting identifiers\n- Using the correct number of arguments for functions\n- Casting to the correct data type\n- Using the proper columns for joins\n\nIf there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.\n\nOutput the final SQL query only.\n\nSQL Query: '), llm=AzureChatOpenAI(verbose=True, client=<openai.resources.chat.completions.Completions object at 0x7a9b6f50cac0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7a9b6f64a290>, model_name='gpt-35-turbo', temperature=0.0, openai_api_key='d7ca6573f74d4b97a97b9f8e58e4d75b', openai_proxy='', request_timeout=45.0, streaming=True, max_tokens=1000, azure_endpoint='https://opnai-genai-dataanlytics-sb.openai.azure.com/', deployment_name='mdl_gpt35_turbo_1106-wmp_genai_dataanalytics-sb', openai_api_version='2024-03-01-preview', openai_api_type='azure')))]
'''

# context is found here
# https://api.python.langchain.com/en/latest/_modules/langchain_community/utilities/sql_database.html#SQLDatabase.get_context
#######
import streamlit as st
from pathlib import Path
import os, json
from langchain_community.chat_models.azure_openai import AzureChatOpenAI #deprecated class, fix later
from langchain.agents import create_sql_agent
from langchain.sql_database import SQLDatabase #from langchain_community.utilities.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
# from langchain.schema import (
#     AIMessage,
#     HumanMessage,
#     SystemMessage,
#     ChatMessage
# )
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import urllib
from loguru import logger

LANGCHAIN_PROJECT = "experiment-5-chat-and-execute-sql-with-WAB"
st.set_page_config(page_title=LANGCHAIN_PROJECT, page_icon="")
st.title(LANGCHAIN_PROJECT)

def load_schema_from_file(file):
    try:
        with open(file, 'r') as f:
            schema = f.read()
        assert schema is not None
        return schema

    except Exception as e:
        logger.error(e)

def run_azure_config():
    config_dir = Path.cwd()
    openai_config_file_path = config_dir / "allconfig.json"
    config_files = [openai_config_file_path]
    config = {}
    for file in config_files:
        with open(file) as json_config:
            config.update(json.load(json_config))
    for k in config:
        os.environ[k] = config[k]

    os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

# def get_wab_connection_string(db_config_file="dbconfig.json"):
#     driver= '{ODBC Driver 18 for SQL Server}'
#     db_config_path = Path.cwd() / db_config_file

#     with open(db_config_path) as json_file:
#         dbconfig = json.load(json_file)

#     server = dbconfig['server']
#     database = dbconfig['database']
#     uid = dbconfig['username']
#     pwd = dbconfig['password']
#     port = int(dbconfig.get("port",1433))
#     pyodbc_connection_string = f"DRIVER={driver};SERVER={server};PORT={port};DATABASE={database};UID={uid};PWD={pwd};Encrypt=yes;Connection Timeout=30;READONLY=True;"
#     params = urllib.parse.quote_plus(pyodbc_connection_string)
#     sqlalchemy_connection_string = f"mssql+pyodbc:///?odbc_connect={params}"
#     return sqlalchemy_connection_string


run_azure_config()


with st.sidebar:
    text = st.text("SQL QUERIES WILL GO HERE")
    SCHEMA_FILEPATH  = (Path.cwd()) / "DDL_for_LLM_upload.sql"
    uploaded_schema = f"{load_schema_from_file(SCHEMA_FILEPATH)}"
    # schema = st.text(f"{uploaded_schema}")

# Setup agent
llm = AzureChatOpenAI(
            temperature=0,
            streaming=True,
            max_tokens=1000,
            azure_deployment=os.environ["AZURE_OPENAI_API_DEPLOYMENT_NAME_GPT35"],
            azure_endpoint=os.environ["AZURE_OPENAI_API_ENDPOINT"],
            model_name=os.environ["MODEL_NAME_GPT35"],
            openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
            request_timeout=45,
            verbose=True,
            #callbacks=[stream_handler] PUSH THE HANDLING OF CALLBACKS TO THE AGENT
        )


@st.cache_resource(ttl="2h")
def get_db_engine(db_config_file="dbconfig.json"):

    def get_wab_connection_string(db_config_file=db_config_file):
        driver= '{ODBC Driver 18 for SQL Server}'
        db_config_path = Path.cwd() / db_config_file

        with open(db_config_path) as json_file:
            dbconfig = json.load(json_file)

        server = dbconfig['server']
        database = dbconfig['database']
        uid = dbconfig['username']
        pwd = dbconfig['password']
        port = int(dbconfig.get("port",1433))
        pyodbc_connection_string = f"DRIVER={driver};SERVER={server};PORT={port};DATABASE={database};UID={uid};PWD={pwd};Encrypt=yes;Connection Timeout=30;READONLY=True;"
        params = urllib.parse.quote_plus(pyodbc_connection_string)
        sqlalchemy_connection_string = f"mssql+pyodbc:///?odbc_connect={params}"
        return sqlalchemy_connection_string

    return SQLDatabase.from_uri(database_uri=get_wab_connection_string(),
                               engine_args={"schema":"sandbox"}
                               )

db=get_db_engine()
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type="openai-tools",
)

with st.sidebar:
    dialect = st.text(f"dialect is {db.dialect}")
    table_names = st.text(f"usable_table names are {db.get_usable_table_names()}")
    table_info = st.text(f"Table info is: {db.get_table_info_no_throw()}")
    q = """
    SELECT TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA = 'sandbox'
    ORDER BY TABLE_NAME;
    """
    sample = st.text('sample query -- db.run(q)')
    result = st.text(f"{db.run(q)}")
    context = st.text(f"Your context is {toolkit.get_context()}")
    tools = st.text(f" Your tools are {toolkit.get_tools()}")




if "messages" not in st.session_state or st.sidebar.button("Clear message history"):
    # see https://blog.langchain.dev/llms-and-sql/
    system_message = f"""
    You are an expert at writing Mircosoft SQL database queries and T-SQL code.
      When asked to write SQL queries use the following schema
     \n\n\n
    {uploaded_schema}
     \n\n\n
     After writing a query, score its estimated accuracy in answering the
      user's question on a scale of 1-5, with 1 the lowest score and 5 the
      highest possible. Respond with the query and the accuracy score. If you give
      an accuracy score of 1 or 2, briefly state your reason.
     """
    st.session_state["messages"] = [{"role": "system", "content":system_message},
                                    {"role": "assistant", "content": "How can I help you?"}
                                    ]
for msg in st.session_state.messages:
    if msg["role"] != "system":
        st.chat_message(msg["role"]).write(msg["content"])

user_query = st.chat_input(placeholder="Ask me anything!")

if user_query:
    st.session_state.messages.append({"role": "user", "content": user_query})
    st.chat_message("user").write(user_query)

    with st.chat_message("assistant"):
        st_cb = StreamlitCallbackHandler(st.container())
        response = agent.invoke(user_query, callbacks=[st_cb])
        st.session_state.messages.append({"role": "assistant", "content": response})
        st.write(response)


Overwriting streaming_app.py


In [66]:
%%writefile streaming_app.py
# experiment-5-chat-and-execute-sql-with-WAB SUCCESS
# reference docs https://python.langchain.com/docs/integrations/toolkits/sql_database#use-sqldatabasetoolkit-within-an-agenthttps://python.langchain.com/docs/expression_language/cookbook/sql_db

#https://python.langchain.com/docs/expression_language/cookbook/sql_db

# context is found here
# https://api.python.langchain.com/en/latest/_modules/langchain_community/utilities/sql_database.html#SQLDatabase.get_context
#######
import streamlit as st
from pathlib import Path
import os, json
from langchain_community.chat_models.azure_openai import AzureChatOpenAI #deprecated class, fix later
from langchain.agents import create_sql_agent
from langchain.sql_database import SQLDatabase #from langchain_community.utilities.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
# from langchain.schema import (
#     AIMessage,
#     HumanMessage,
#     SystemMessage,
#     ChatMessage
# )
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import urllib
from loguru import logger

LANGCHAIN_PROJECT = "experiment-5-chat-and-execute-sql-with-WAB"
st.set_page_config(page_title=LANGCHAIN_PROJECT, page_icon="")
st.title(LANGCHAIN_PROJECT)

def load_schema_from_file(file):
    try:
        with open(file, 'r') as f:
            schema = f.read()
        assert schema is not None
        return schema

    except Exception as e:
        logger.error(e)

def run_azure_config():
    config_dir = Path.cwd()
    openai_config_file_path = config_dir / "allconfig.json"
    config_files = [openai_config_file_path]
    config = {}
    for file in config_files:
        with open(file) as json_config:
            config.update(json.load(json_config))
    for k in config:
        os.environ[k] = config[k]

    os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

# def get_wab_connection_string(db_config_file="dbconfig.json"):
#     driver= '{ODBC Driver 18 for SQL Server}'
#     db_config_path = Path.cwd() / db_config_file

#     with open(db_config_path) as json_file:
#         dbconfig = json.load(json_file)

#     server = dbconfig['server']
#     database = dbconfig['database']
#     uid = dbconfig['username']
#     pwd = dbconfig['password']
#     port = int(dbconfig.get("port",1433))
#     pyodbc_connection_string = f"DRIVER={driver};SERVER={server};PORT={port};DATABASE={database};UID={uid};PWD={pwd};Encrypt=yes;Connection Timeout=30;READONLY=True;"
#     params = urllib.parse.quote_plus(pyodbc_connection_string)
#     sqlalchemy_connection_string = f"mssql+pyodbc:///?odbc_connect={params}"
#     return sqlalchemy_connection_string


run_azure_config()

# Setup agent
llm = AzureChatOpenAI(
            temperature=0,
            streaming=True,
            max_tokens=1000,
            azure_deployment=os.environ["AZURE_OPENAI_API_DEPLOYMENT_NAME_GPT35"],
            azure_endpoint=os.environ["AZURE_OPENAI_API_ENDPOINT"],
            model_name=os.environ["MODEL_NAME_GPT35"],
            openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
            request_timeout=45,
            verbose=True,
            #callbacks=[stream_handler] PUSH THE HANDLING OF CALLBACKS TO THE AGENT
        )


@st.cache_resource(ttl="2h")
def get_db_engine(db_config_file="dbconfig.json"):

    def get_wab_connection_string(db_config_file=db_config_file):
        driver= '{ODBC Driver 18 for SQL Server}'
        db_config_path = Path.cwd() / db_config_file

        with open(db_config_path) as json_file:
            dbconfig = json.load(json_file)

        server = dbconfig['server']
        database = dbconfig['database']
        uid = dbconfig['username']
        pwd = dbconfig['password']
        port = int(dbconfig.get("port",1433))
        pyodbc_connection_string = f"DRIVER={driver};SERVER={server};PORT={port};DATABASE={database};UID={uid};PWD={pwd};Encrypt=yes;Connection Timeout=30;READONLY=True;"
        params = urllib.parse.quote_plus(pyodbc_connection_string)
        sqlalchemy_connection_string = f"mssql+pyodbc:///?odbc_connect={params}"
        return sqlalchemy_connection_string

    kwargs = {"schema":"sandbox"}
    return SQLDatabase.from_uri(database_uri=get_wab_connection_string(),
                               **kwargs
                               )

db=get_db_engine()
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type="openai-tools",
)

with st.sidebar:
    # text = st.text("SQL QUERIES WILL GO HERE")
    SCHEMA_FILEPATH  = (Path.cwd()) / "DDL_for_LLM_upload.sql"
    uploaded_schema = f"{load_schema_from_file(SCHEMA_FILEPATH)}"
    # schema = st.text(f"{uploaded_schema}")
#    dialect = st.text(f"dialect is {db.dialect}")
#    table_names = st.text(f"usable_table names are {db.get_usable_table_names()}")
#    table_info = st.text(f"Table info is: {db.get_table_info_no_throw()}")
    # q = """
    # SELECT TABLE_NAME
    # FROM INFORMATION_SCHEMA.TABLES
    # WHERE TABLE_SCHEMA = 'sandbox'
    # ORDER BY TABLE_NAME;
    # """
    # sample = st.text('sample query -- db.run(q)')
    # result = st.text(f"{db.run(q)}")
    context = st.text(f"Your context is {toolkit.get_context()}")
    tools = st.text(f" Your tools are {toolkit.get_tools()}")




if "messages" not in st.session_state or st.sidebar.button("Clear message history"):
    # see https://blog.langchain.dev/llms-and-sql/
    system_message = f"""
    You are an expert at writing Mircosoft SQL database queries and T-SQL code.
      When asked to write SQL queries use the following schema
     \n\n\n
    {uploaded_schema}
     \n\n\n
     After writing a query, score its estimated accuracy in answering the
      user's question on a scale of 1-5, with 1 the lowest score and 5 the
      highest possible. Respond with the query and the accuracy score. If you give
      an accuracy score of 1 or 2, briefly state your reason.
     """
    st.session_state["messages"] = [{"role": "system", "content":system_message},
                                    {"role": "assistant", "content": "How can I help you?"}
                                    ]
for msg in st.session_state.messages:
    if msg["role"] != "system":
        st.chat_message(msg["role"]).write(msg["content"])

user_query = st.chat_input(placeholder="Ask me anything!")

if user_query:
    st.session_state.messages.append({"role": "user", "content": user_query})
    st.chat_message("user").write(user_query)

    with st.chat_message("assistant"):
        st_cb = StreamlitCallbackHandler(st.container())
        response = agent.invoke(user_query, callbacks=[st_cb])
        st.session_state.messages.append({"role": "assistant", "content": response})
        st.write(response)


Overwriting streaming_app.py


In [76]:
%%writefile streaming_app.py
# experiment-5-chat-and-execute-sql-with-WAB FAILURE
# reference docs https://python.langchain.com/docs/integrations/toolkits/sql_database#use-sqldatabasetoolkit-within-an-agenthttps://python.langchain.com/docs/expression_language/cookbook/sql_db

#https://python.langchain.com/docs/expression_language/cookbook/sql_db




##### this function is garbage
#[docs]    def get_usable_table_names(self) -> Iterable[str]:
#        """Get names of tables available."""
#        if self._include_tables:
#            return sorted(self._include_tables)
#        return sorted(self._all_tables - self._ignore_tables)
#####
'''
Your context is {'table_info': '\nCREATE TABLE [TELEPHONE_TYPE] (\n\tcolumn1 NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, \n\tcolumn2 NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL\n)\n\n/*\n3 rows from TELEPHONE_TYPE table:\ncolumn1\tcolumn2\nL\tLandline\nM\tMobile\nF\tFax\n*/', 'table_names': 'TELEPHONE_TYPE'}

Your tools are [QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a9b5f0a6680>), InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a9b5f0a6680>), ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a9b5f0a6680>), QuerySQLCheckerTool(description='Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a9b5f0a6680>, llm=AzureChatOpenAI(verbose=True, client=<openai.resources.chat.completions.Completions object at 0x7a9b6f50cac0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7a9b6f64a290>, model_name='gpt-35-turbo', temperature=0.0, openai_api_key='d7ca6573f74d4b97a97b9f8e58e4d75b', openai_proxy='', request_timeout=45.0, streaming=True, max_tokens=1000, azure_endpoint='https://opnai-genai-dataanlytics-sb.openai.azure.com/', deployment_name='mdl_gpt35_turbo_1106-wmp_genai_dataanalytics-sb', openai_api_version='2024-03-01-preview', openai_api_type='azure'), llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['dialect', 'query'], template='\n{query}\nDouble check the {dialect} query above for common mistakes, including:\n- Using NOT IN with NULL values\n- Using UNION when UNION ALL should have been used\n- Using BETWEEN for exclusive ranges\n- Data type mismatch in predicates\n- Properly quoting identifiers\n- Using the correct number of arguments for functions\n- Casting to the correct data type\n- Using the proper columns for joins\n\nIf there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.\n\nOutput the final SQL query only.\n\nSQL Query: '), llm=AzureChatOpenAI(verbose=True, client=<openai.resources.chat.completions.Completions object at 0x7a9b6f50cac0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7a9b6f64a290>, model_name='gpt-35-turbo', temperature=0.0, openai_api_key='d7ca6573f74d4b97a97b9f8e58e4d75b', openai_proxy='', request_timeout=45.0, streaming=True, max_tokens=1000, azure_endpoint='https://opnai-genai-dataanlytics-sb.openai.azure.com/', deployment_name='mdl_gpt35_turbo_1106-wmp_genai_dataanalytics-sb', openai_api_version='2024-03-01-preview', openai_api_type='azure')))]
'''

# context is found here
# https://api.python.langchain.com/en/latest/_modules/langchain_community/utilities/sql_database.html#SQLDatabase.get_context
#######
import streamlit as st
from pathlib import Path
import os, json
from langchain_community.chat_models.azure_openai import AzureChatOpenAI #deprecated class, fix later
from langchain.agents import create_sql_agent
from langchain.sql_database import SQLDatabase #from langchain_community.utilities.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
# from langchain.schema import (
#     AIMessage,
#     HumanMessage,
#     SystemMessage,
#     ChatMessage
# )
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import urllib
from loguru import logger

LANGCHAIN_PROJECT = "experiment-5-chat-and-execute-sql-with-WAB"
st.set_page_config(page_title=LANGCHAIN_PROJECT, page_icon="")
st.title(LANGCHAIN_PROJECT)

def load_schema_from_file(file):
    try:
        with open(file, 'r') as f:
            schema = f.read()
        assert schema is not None
        return schema

    except Exception as e:
        logger.error(e)

def run_azure_config():
    config_dir = Path.cwd()
    openai_config_file_path = config_dir / "allconfig.json"
    config_files = [openai_config_file_path]
    config = {}
    for file in config_files:
        with open(file) as json_config:
            config.update(json.load(json_config))
    for k in config:
        os.environ[k] = config[k]

    os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

# def get_wab_connection_string(db_config_file="dbconfig.json"):
#     driver= '{ODBC Driver 18 for SQL Server}'
#     db_config_path = Path.cwd() / db_config_file

#     with open(db_config_path) as json_file:
#         dbconfig = json.load(json_file)

#     server = dbconfig['server']
#     database = dbconfig['database']
#     uid = dbconfig['username']
#     pwd = dbconfig['password']
#     port = int(dbconfig.get("port",1433))
#     pyodbc_connection_string = f"DRIVER={driver};SERVER={server};PORT={port};DATABASE={database};UID={uid};PWD={pwd};Encrypt=yes;Connection Timeout=30;READONLY=True;"
#     params = urllib.parse.quote_plus(pyodbc_connection_string)
#     sqlalchemy_connection_string = f"mssql+pyodbc:///?odbc_connect={params}"
#     return sqlalchemy_connection_string


run_azure_config()

# Setup agent
llm = AzureChatOpenAI(
            temperature=0,
            streaming=True,
            max_tokens=1000,
            azure_deployment=os.environ["AZURE_OPENAI_API_DEPLOYMENT_NAME_GPT35"],
            azure_endpoint=os.environ["AZURE_OPENAI_API_ENDPOINT"],
            model_name=os.environ["MODEL_NAME_GPT35"],
            openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
            request_timeout=45,
            verbose=True,
            #callbacks=[stream_handler] PUSH THE HANDLING OF CALLBACKS TO THE AGENT
        )


@st.cache_resource(ttl="2h")
def get_db_engine(db_config_file="dbconfig.json"):

    def get_wab_connection_string(db_config_file=db_config_file):
        driver= '{ODBC Driver 18 for SQL Server}'
        db_config_path = Path.cwd() / db_config_file

        with open(db_config_path) as json_file:
            dbconfig = json.load(json_file)

        server = dbconfig['server']
        database = dbconfig['database']
        uid = dbconfig['username']
        pwd = dbconfig['password']
        port = int(dbconfig.get("port",1433))
        pyodbc_connection_string = f"DRIVER={driver};SERVER={server};PORT={port};DATABASE={database};UID={uid};PWD={pwd};Encrypt=yes;Connection Timeout=30;READONLY=True;"
        params = urllib.parse.quote_plus(pyodbc_connection_string)
        sqlalchemy_connection_string = f"mssql+pyodbc:///?odbc_connect={params}"
        return sqlalchemy_connection_string

    kwargs = {"schema":"sandbox"}
    return SQLDatabase.from_uri(get_wab_connection_string(),**kwargs)

db=get_db_engine()
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type="openai-tools",
)

with st.sidebar:
    # text = st.text("SQL QUERIES WILL GO HERE")
    SCHEMA_FILEPATH  = (Path.cwd()) / "DDL_for_LLM_upload.sql"
    uploaded_schema = f"{load_schema_from_file(SCHEMA_FILEPATH)}"
    # schema = st.text(f"{uploaded_schema}")
    dialect = st.text(f"dialect is {db.dialect}")
    all_tables = st.text(f"all tables = {db._all_tables}")
    include_tables = st.text(f"include tables = {db._include_tables}")
    ignore_tables = st.text(f"ignore tables = {db._ignore_tables}")
    table_names = st.text(f"usable_table names are {list(db.get_usable_table_names())}")
    table_info = st.text(f"Table info is: {db.get_table_info_no_throw()}")
    # q = """
    # SELECT TABLE_NAME
    # FROM INFORMATION_SCHEMA.TABLES
    # WHERE TABLE_SCHEMA = 'sandbox'
    # ORDER BY TABLE_NAME;
    # """
    # sample = st.text('sample query -- db.run(q)')
    # result = st.text(f"{db.run(q)}")
    # context = st.text(f"Your context is {toolkit.get_context()}")
    # tools = st.text(f" Your tools are {toolkit.get_tools()}")




if "messages" not in st.session_state or st.sidebar.button("Clear message history"):
    # see https://blog.langchain.dev/llms-and-sql/
    system_message = f"""
    You are an expert at writing Mircosoft SQL database queries and T-SQL code.
      When asked to write SQL queries use the following schema
     \n\n\n
    {uploaded_schema}
     \n\n\n
     After writing a query, score its estimated accuracy in answering the
      user's question on a scale of 1-5, with 1 the lowest score and 5 the
      highest possible. Respond with the query and the accuracy score. If you give
      an accuracy score of 1 or 2, briefly state your reason.
     """
    st.session_state["messages"] = [{"role": "system", "content":system_message},
                                    {"role": "assistant", "content": "How can I help you?"}
                                    ]
for msg in st.session_state.messages:
    if msg["role"] != "system":
        st.chat_message(msg["role"]).write(msg["content"])

user_query = st.chat_input(placeholder="Ask me anything!")

if user_query:
    st.session_state.messages.append({"role": "user", "content": user_query})
    st.chat_message("user").write(user_query)

    with st.chat_message("assistant"):
        st_cb = StreamlitCallbackHandler(st.container())
        response = agent.invoke(user_query, callbacks=[st_cb])
        st.session_state.messages.append({"role": "assistant", "content": response})
        st.write(response)


Overwriting streaming_app.py


In [77]:
!streamlit run streaming_app.py &>/content/logs.txt &
!curl ipv4.icanhazip.com
!echo "Copy the IP above into the webpage that opens below"
!echo "you also need to allow the port access to the database"
!npx localtunnel --port 8501


35.221.32.163
Copy the IP above into the webpage that opens below
you also need to allow the port access to the database
npx: installed 22 in 2.194s
your url is: https://tasty-shoes-say.loca.lt
^C
